# オンライン分析用  

本ノートブックは、収集したログ情報から、最新の実行履歴を確認するサンプルを記載しています。

## 表示

出力例を掲載しています。

- ノートブックごとに、表形式で表示します
- 各ユーザごとに、各セルの最新の実行結果が表示されます
  - セルの実行結果が`ok`（正常終了）の場合は背景色緑、セルの実行結果が`error`（異常終了）の場合は背景色赤で表示しています
  - 一度も実行していないか、実行状況を把握できないセルには何も表示されません。
<img src="./images/progress_html.png" width=300 height=300>

## パラメータ指定

対象の課題を指定します。  
以下のセルを実行するとドロップダウンメニューが表示されるため、課題を選択してください。

In [ ]:
import os

from IPython.display import display
from ipywidgets import widgets
from nbgrader.api import Gradebook, MissingEntry

db_path = os.path.join(os.environ['HOME'], 'nbgrader', os.environ['MOODLECOURSE'], 'gradebook.db')
gb = Gradebook(f'sqlite:///{db_path}',
               os.environ['MOODLECOURSE'])

assignment_selection = widgets.Dropdown(
    options = [d.name for d in gb.assignments],
    description = "課題を選択してください:",
)
display(assignment_selection)

In [ ]:
import os

# Control Panel → Token より発行したトークンを入力してください
TOKEN = 

# -- 以下は必要に応じて変更してください 基本的には変更不要です。--
OUTPUT_DIR = 'output' # 進捗確認用HTMLファイルの出力先
COURSE = os.environ['MOODLECOURSE']
COURSE_PATH = os.path.join(os.environ['HOME'], 'nbgrader', COURSE)
SOURCE_PATH = os.path.join(COURSE_PATH, 'source')
DB_PATH = os.path.join(COURSE_PATH, 'exec_history.db')
ASSIGNMENT = assignment_selection.value

## 出力

出力したHTMLファイルをブラウザで閲覧するか、このノートブック上で閲覧する方法があります。

### html更新用の処理  

教師用の元ファイルを読み込みます。  
また、必要なディレクトリを作成します。

In [ ]:
src = os.path.join(SOURCE_PATH, ASSIGNMENT, '*.ipynb')
!jupyter nbconvert --to html --output-dir {'html/'+ASSIGNMENT} {src}

if not os.path.isdir(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

htmlファイルに最新のDBの内容を取り込む処理を実装しています。  
ここに定義した関数を実行するたびに、DBから最新の情報を取得し、各ユーザの実行状況をHTMLとして出力します。

In [ ]:
import os

from datetime import datetime
import requests
import sqlite3
from jinja2 import Environment, FileSystemLoader


def log2db(token, course, assignment, dt_from: datetime = None):
    """学生の実行ログ情報をDBに収集する
    """

    headers = {"Content-Type": "application/json",
               "Authorization": f"token {token}"}
    data = {
        'course': course,
        'assignment': assignment,
    }
    if dt_from is not None:
        data['from'] = dt_from.isoformat()
    r = requests.post('http://jupyterhub:8088/services/teachertools/api/log_collect',
                      headers=headers,
                      json=data)
    return r


def update_progress_html(fname: str, course: str, assignment: str,
                         html_auto_refresh_sec: int = -1):
    """ログ情報をDBから読み込み、HTMLファイルに出力する
    """
    file_loader = FileSystemLoader('./')
    env = Environment(loader=file_loader)
    template = env.get_template('template/progress.html.j2')
    
    sql = """
    SELECT
      notebook_name,
      cell.section as cell_section,
      ifnull(log_executed.student_id, null) as student_id,
      cell.id as cell_id,
      cell.jupyter_cell_id as jupyter_cell_id,
      ifnull(log_executed.log_execute_reply_status, '') as log_execute_reply_status
    FROM
      cell left outer join (SELECT
          log.assignment as assignment,
          log.student_id as student_id,
          log.cell_id as cell_id,
          log_execute_reply_status
        FROM log, 
            (SELECT assignment, student_id, cell_id, MAX(log_sequence) as sequence
            FROM log
            WHERE assignment = ?
            GROUP BY
              student_id, cell_id) latest_log
        where log.assignment = latest_log.assignment
        and log.student_id = latest_log.student_id
        and log.cell_id = latest_log.cell_id
        and log.log_sequence = latest_log.sequence
        order by
          log.cell_id, student_id) log_executed on cell.id = log_executed.cell_id and cell.assignment = log_executed.assignment
    WHERE
      cell.assignment = ?
    ORDER BY
      cell.notebook_name, cell.id
    """
    student_sql = """
    SELECT id
    FROM student
    """

    with sqlite3.connect(f"file:{DB_PATH}?mode=ro", uri=True) as con:
        con.row_factory = sqlite3.Row
        cur = con.cursor()
        cur.execute(sql, [assignment, assignment])
        result = cur.fetchall()
        cur.execute(student_sql)
        students = [d['id'] for d in cur.fetchall()]

    notebooks = dict()
    c_notebook_name = None
    c_cell_id = None
    c_idx = 0
    nb_changed = False
    for r in result:
        notebook_name = r['notebook_name']
        cell_id = r['cell_id']
        jupyter_cell_id = r['jupyter_cell_id']
        cell_section = r['cell_section']
        student_id = r['student_id']
        log_execute_reply_status = r['log_execute_reply_status']

        if c_notebook_name != notebook_name:
            notebooks[notebook_name] = []
            c_notebook_name = notebook_name
            c_idx = 0
            nb_changed = True

        if c_cell_id != cell_id:
            notebooks[notebook_name].append(dict(cell_id=cell_id, section=cell_section, jupyter_cell_id=jupyter_cell_id))
            c_cell_id = cell_id
            if nb_changed is True:
                nb_changed = False
                # ノートブックが変わった時はインクリメントしない
            else:
                c_idx += 1
        if 'student_results' not in notebooks[notebook_name][c_idx]:
            notebooks[notebook_name][c_idx]['student_results'] = {}

        if student_id is not None:
            notebooks[notebook_name][c_idx]['student_results'][student_id] = {'exec_info': {'state': log_execute_reply_status}}

    data = {
        'title': '進捗可視化',
        'heading': '進捗可視化',
        'course_name': course,
        'assignment_name': assignment,
        'notebooks': notebooks,
        'users': students,
        'html_auto_refresh_sec': html_auto_refresh_sec,
    }
    output = template.render(data)

    with open(fname, 'w') as f:
        f.write(output)

    return fname


## 閲覧  

ブラウザで新たにタブを開いて出力したHTMLを閲覧する方法と、本ノートブックのセル出力でHTMLを閲覧する方法があります。  
自動更新するための手段が異なるため、セルを分けています。

### 別タブで開く場合（自動更新）

このセルの実行結果として、出力したHTMLファイルへのリンクが表示されます。

【補足】  
「1.4.2 セルの出力で開く場合」で表示する場合は、上記自動リロードを設定していると、このノートブックを開いているタブ自体がリロードされてしまいます。これを防ぐために、別タブで開く場合と、セルの実行結果で開く場合でhtmlファイルを分けています。  
そのため、htmlファイルの名前を変更する場合は、同じ名前にならないように設定することを推奨しています。

In [ ]:
from datetime import datetime, timezone, timedelta
import getpass
import time
import os
from pathlib import Path

# 自動更新する間隔（秒）
html_reload_span = 0  # htmlのリロード間隔（秒） 0以下の場合、自動でリロードしない
db_reload_span = 10   # 実行ログ収集間隔（秒） 0以下は設定不可
html_name = f'{OUTPUT_DIR}/{COURSE}_{ASSIGNMENT}_autoreload.html'
user_name = getpass.getuser()

r = log2db(TOKEN, COURSE, ASSIGNMENT)
if r.status_code != 200:
    raise Exception('Status is not ok', r.status_code)
f = update_progress_html(html_name, COURSE, ASSIGNMENT)
f_path = os.path.join(str(Path().resolve()).replace(os.environ["HOME"] + "/", ""), f)
link = f'https://{os.environ["JUPYTERHUB_FQDN"]}/user/{user_name}/files/{f_path}'

print(f"Link: {link}")
while True:
    latest_log_dt = datetime.now(timezone.utc) - timedelta(seconds=db_reload_span)
    log2db(TOKEN, COURSE, ASSIGNMENT, latest_log_dt)
    update_progress_html(html_name, COURSE, ASSIGNMENT, html_auto_refresh_sec=html_reload_span)
    time.sleep(db_reload_span)

### セルの出力で開く場合

セルの出力を自動更新するようになっています。  
以下のセルでは、10秒ごとに自動でリロードを行う設定になっています。  
自動更新する秒数の変更や、自動更新自体行わない場合は、以下のセルを編集してください。  

In [ ]:
import time
from datetime import timezone, timedelta

from IPython.display import HTML

# 自動更新する間隔（秒）
reload_span = 10
html_name = f'{OUTPUT_DIR}/{COURSE}_{ASSIGNMENT}.html'

log2db(TOKEN, COURSE, ASSIGNMENT)
f = update_progress_html(html_name, COURSE, ASSIGNMENT)
display_handle = display(HTML(f), display_id=True, clear=True)

while True:
    latest_log_dt = datetime.now(timezone.utc) - timedelta(seconds=reload_span)
    log2db(TOKEN, COURSE, ASSIGNMENT, latest_log_dt)
    f = update_progress_html(html_name, COURSE, ASSIGNMENT)
    display_handle.update(HTML(f))
    time.sleep(reload_span)